In [1]:
import os
try:
    os.chdir('src')
except:
    pass
print(os.getcwd())

/home/shamvinc/ssl_time_series/mvts_transformer/src


In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


<center><img src="../img/img_0.PNG"  width="1000" height="240"/></center>


# SimMTM is a simple self-supervised learning framework for Masked Time-Series Modeling.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm
import copy



from datasets.datasplit import split_dataset
from datasets.data import data_factory, Normalizer, TSRegressionArchive, CSVRegressionArchive
from datasets.datasplit import split_dataset
from datasets.dataset import collate_superv
from models.ts_transformer import model_factory
from models.loss import get_loss_module, contrastive_loss
from optimizers import get_optimizer

from options import Options
from running import setup


# Masked Modeling

<img src="../img/img_1.PNG"  width="900" height="240"/>

<img src="../img/img_2.PNG"  width="900" height="240"/>

### Random Masking is not a good choice to learn a good representation. 

<img src="../img/img_3.PNG" width="600"/>

### Therefore, we randomly mask a sequence of points in a serie.

In [4]:
def geom_noise_mask_single(L, lm, masking_ratio):
    """
    Randomly create a boolean mask of length `L`, consisting of subsequences of average length lm, masking with 0s a `masking_ratio`
    proportion of the sequence L. The length of masking subsequences and intervals follow a geometric distribution.
    Args:
        L: length of mask and sequence to be masked
        lm: average length of masking subsequences (streaks of 0s)
        masking_ratio: proportion of L to be masked

    Returns:
        (L,) boolean numpy array intended to mask ('drop') with 0s a sequence of length L
    """
    keep_mask = np.ones(L, dtype=bool)
    p_m = 1 / lm  # probability of each masking sequence stopping. parameter of geometric distribution.
    p_u = p_m * masking_ratio / (1 - masking_ratio)  # probability of each unmasked sequence stopping. parameter of geometric distribution.
    p = [p_m, p_u]

    # Start in state 0 with masking_ratio probability
    state = int(np.random.rand() > masking_ratio)  # state 0 means masking, 1 means not masking
    for i in range(L):
        keep_mask[i] = state  # here it happens that state and masking value corresponding to state are identical
        if np.random.rand() < p[state]:
            state = 1 - state

    return keep_mask

# SimMTM ultilizes both contrastive learning and mask modeling to learn the data representation.


# 1 - Contrastive learning

<img src="../img/img_5.png"/>

### The contrastive loss is the following: (Eq. 8 in the paper)

<center><img src="../img/img_6.PNG"/><center/>

In [5]:
def demo_contrastive_loss(z, batch_size):
    s = s.squeeze(-1) 

    B = s.shape[0]
    v = s.reshape(B, -1)

    norm_v = torch.norm(v, p=2, dim=-1).unsqueeze(-1)
    v = v/norm_v
    u = torch.transpose(v, 0, 1)

    R = torch.matmul(v,u)

 
    R = torch.exp(R/tau) # (batch + mask size) x (batch + mask size)
    
    # number of masks
    M = B//batch_size
    mask = torch.eye(batch_size, device=R.device).repeat_interleave(M,dim=0).repeat_interleave(M,dim=1)

    denom = R * (torch.ones_like(R) - torch.eye(R.shape[0], device=R.device))

    denom = R.sum(-1).unsqueeze(-1)

    loss = torch.log(R/denom)
    

    loss = (loss * (mask - torch.eye(R.shape[0], device=R.device))).sum(1)/(M-1) # except no masked unit
    loss = loss.mean(0)
    
    return -loss


# 2 - Masked Modeling

### SimMTM proposes to recover a time serie by the weighted sum of multiple masked points, which eases the reconstruction task by assembling ruined but complementary temporal variations.

<img src="../img/img_4.png"/>

In [6]:


class DemoSimMTMTransformerEncoder(nn.Module):

    def forward(self, X):
        """
        Args:
            X: (batch_size, seq_length, feat_dim) torch tensor of masked features (input)
            padding_masks: (batch_size, seq_length) boolean tensor, 1 means keep vector at this position, 0 means padding
        Returns:
            output: (batch_size, seq_length, feat_dim)
        """

        # permute because pytorch convention for transformers is [seq_length, batch_size, feat_dim]. padding_masks [batch_size, feat_dim]
        _x = X
     
        for i in range(self.temporal_unit):
            mask = geom_noise_mask_single(X.shape[0] * X.shape[1] * X.shape[2], 5, 0.3)
            mask = mask.reshape(X.shape[0], X.shape[1], X.shape[2])
            mask = torch.from_numpy(mask).to(X.device)
            x_masked = mask * X
            _x = torch.cat([_x, x_masked], axis=-1) # [batch_size, seq_length, feat_dim * temporal_unit]
    
        
        _x = _x.reshape(X.shape[0] * (self.temporal_unit + 1), X.shape[1], X.shape[2])
  

        inp = _x.permute(1, 0, 2)
        inp = self.project_inp(inp) * math.sqrt(self.d_model)  # [seq_length, batch_size, d_model] project input vectors to d_model dimensional space
        inp = self.pos_enc(inp)  # add positional encoding

        
        output = self.transformer_encoder(inp)  # (seq_length, batch_size, d_model)
        output = self.act(output)  # the output transformer encoder/decoder embeddings don't include non-linearity
        output = output.permute(1, 0, 2)  # (batch_size, seq_length, d_model)
        output = self.dropout1(output)

        z_hat = self.project(output)
        # Most probably defining a Linear(d_model,feat_dim) vectorizes the operation over (seq_length, batch_size).
        output = self.output_layer(z_hat)  # (batch_size, seq_length, feat_dim)

        return output

    
    def project(self, z, tau=0.02):
        _z = z.transpose(1, 2) # [batch_size, d_model, seq_length]
        s = self.projector_layer(_z) # [batch_size, d_model, 1]
        s = s.squeeze(-1) 

        B = s.shape[0]
        v = s.reshape(B, -1)

        norm_v = torch.norm(v, p=2, dim=-1).unsqueeze(-1)
        v = v/norm_v
        u = torch.transpose(v, 0, 1)
        
        R = torch.matmul(v,u)
     
  
        R = torch.exp(R/tau) # (batch + mask size) x (batch + mask size)
        R = R * (torch.ones_like(R) - torch.eye(R.shape[0], device=R.device)) # zero out the weight of no masked component
        R = R/R.sum(-1).unsqueeze(-1)
        M = self.temporal_unit + 1
        R = R[::M] # extract every no mask unit # (batch size) x (batch + mask size)

        z_hat = (R.unsqueeze(-1).unsqueeze(-1) * z.unsqueeze(0)).sum(1) 
        return z_hat




# Data Loading and Preparation

In [7]:
args = Options().parse()  # `argsparse` object
args.data_dir = '../datasets/BeijingPM25Quality'
args.task = 'regression'
args.output_dir = '../experiments'
config = setup(args)
# config = setup(args)  # configuration dictionary

2023-08-23 03:25:16,988 | INFO : Stored configuration file in '../experiments/_2023-08-23_03-25-16_GlX'


In [8]:
data = TSRegressionArchive(config['data_dir'], pattern='TRAIN', config=config)
test_data = TSRegressionArchive(config['data_dir'], pattern='TEST', config=config)

11942it [00:48, 245.72it/s]
5072it [00:20, 252.53it/s]


In [36]:
data.feature_df.loc[11917]

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8
11917,7.0,59.0,800.0,14.0,-1.4,1030.2,-11.3,0.0,0.9
11917,8.0,59.0,900.0,11.0,-1.6,1030.2,-10.2,0.0,0.8
11917,7.0,57.0,900.0,11.0,-1.6,1030.4,-10.7,0.0,1.6
11917,6.0,59.0,900.0,9.0,-3.0,1030.5,-10.3,0.0,1.1
11917,5.0,53.0,800.0,15.0,-2.4,1030.4,-10.5,0.0,1.2
11917,5.0,39.0,600.0,29.0,-3.7,1030.1,-10.8,0.0,0.7
11917,6.0,47.0,600.0,18.0,-5.8,1030.0,-10.6,0.0,1.0
11917,4.0,52.0,600.0,15.0,-5.5,1030.2,-11.0,0.0,0.7
11917,4.0,61.0,800.0,8.0,-5.1,1030.2,-10.9,0.0,1.0
11917,4.0,58.0,800.0,12.0,-3.0,1030.7,-10.8,0.0,1.5


In [10]:
data.labels_df

,0
0,24.0
1,93.0
2,117.0
3,58.0
4,226.0
...,...
11913,89.0
11914,281.0
11915,543.0
11916,505.0


In [10]:
train_indices, val_indices, _ = split_dataset(data_indices=data.all_IDs,
                                                         validation_method='ShuffleSplit',
                                                         n_splits=1,
                                                         validation_ratio=0.2,
                                                         test_set_ratio=0,  # used only if test_indices not explicitly specified
                                                         test_indices=None,
                                                         random_seed=1337,
                                                         labels=None)
train_indices = train_indices[0]
val_indices = val_indices[0]
test_indices = np.array(test_data.all_IDs)

In [11]:
normalizer = Normalizer(config['normalization'])
data.feature_df = normalizer.normalize(data.feature_df)
test_data.feature_df = normalizer.normalize(test_data.feature_df)

In [37]:
config['task'] = 'simmtm'
config['normalization_layer'] = 'BatchNorm'
config['out_len'] = 1
config['out_dim'] = 1
config['d_model'] = 8
config['dim_feedforward'] = 32
config['num_heads'] = 4
config['num_layers'] = 2
# SimMTMTransformerEncoder
from models.ts_transformer import model_factory
model = model_factory(config, data)
device = "cuda"
model.to(device)
model.tau = 0.05
min_tau = 0.05
model.mask_length = data.feature_df.loc[0].shape[0]//2
model.mask_ratio = .5

In [38]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [39]:
from torch.utils.data import DataLoader
batch_size = 64
max_len = 24
train_dataloader = DataLoader(train_indices, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_indices, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_indices, batch_size=batch_size, shuffle=True)



# Pre-Training Loop

In [40]:

i = 0
max_epoch = 0
best_loss = 1e10
best_model = copy.deepcopy(model)
best_epoch = 0
loss_fn = nn.MSELoss()


while i < max_epoch:
    train_loss = { "loss": [], "loss_mse": [], "loss_con": []}
    progress_bar = tqdm(train_dataloader)
    
    for IDs in progress_bar:
        model.train()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.float()
        X = X.reshape(-1, max_len, X.shape[-1])
        
        pred, R, w1, w2, norm = model(X)  # (batch_size, padded_length, feat_dim)
        
        loss_mse = loss_fn(pred, X) 

        loss_con = contrastive_loss(R, X.shape[0])
        
        loss_norm =  norm.mean()

        loss = 1/(w1.pow(2)) * loss_mse + 1/(w2.pow(2)) * loss_con + torch.log(w1) + torch.log(w2)
  


        optimizer.zero_grad()
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
        optimizer.step()

        progress_bar.set_description("Epoch {0} - Training loss: {1:.2f} - MSE loss: {2:.2f} - Contrastive loss: {3:.2f} - norm loss: {3:.2f}".format(i, 
                loss.cpu().detach().numpy().item(), loss_mse.cpu().detach().numpy().item(), 
                loss_con.cpu().detach().numpy().item(), loss_norm.cpu().detach().numpy().item())) 
        train_loss["loss"].append(loss)
        train_loss["loss_mse"].append(loss_mse)
        train_loss["loss_con"].append(loss_con)
    model.tau = max(model.tau * 0.98, min_tau)
        
    
    val_loss = { "loss": [], "loss_mse": [], "loss_con": []}
    for IDs in val_dataloader:
        model.eval()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.float()
        X = X.reshape(-1, max_len, X.shape[-1])
        
                
        pred, R, w1, w2, norm = model(X.float())  # (batch_size, padded_length, feat_dim)
        
        loss_mse = loss_fn(pred, X) 

        loss_con = contrastive_loss(R, X.shape[0])

        loss = 1/(w1.pow(2)) * loss_mse + 1/(w2.pow(2)) * loss_con + torch.log(w1) + torch.log(w2)
        
        val_loss["loss"].append(loss)
        val_loss["loss_mse"].append(loss_mse)
        val_loss["loss_con"].append(loss_con)
    
    train_loss["loss"] = torch.tensor(train_loss["loss"]).mean()
    train_loss["loss_mse"] = torch.tensor(train_loss["loss_mse"]).mean()
    train_loss["loss_con"] = torch.tensor(train_loss["loss_con"]).mean()
    val_loss["loss"] = torch.tensor(val_loss["loss"]).mean()
    val_loss["loss_mse"] = torch.tensor(val_loss["loss_mse"]).mean()
    val_loss["loss_con"] = torch.tensor(val_loss["loss_con"]).mean()
    
    if val_loss["loss"] < best_loss and model.tau == min_tau:
        best_loss = val_loss["loss"]
        best_model = copy.deepcopy(model)
        best_epoch = i
    
    progress_bar.write("Epoch {0} - Training loss: {1:.2f} {2:.2f} {3:.2f} - Validation loss: {4:.2f} {5:.2f} {6:.2f}".format(i, 
            train_loss["loss"].cpu().detach().numpy().item(), train_loss["loss_mse"].cpu().detach().numpy().item(), train_loss["loss_con"].cpu().detach().numpy().item(),
            val_loss["loss"].cpu().detach().numpy().item(), val_loss["loss_mse"].cpu().detach().numpy().item(), val_loss["loss_con"].cpu().detach().numpy().item()))
    i += 1
    
    
tqdm.write("Best Epoch {} - Best Validation loss: {}".format(best_epoch, best_loss))

Best Epoch 0 - Best Validation loss: 10000000000.0


# Replace the output layer for downstream task

In [41]:
finetune_model = copy.deepcopy(best_model)
# finetune_model.predict_layer1 = nn.Linear(finetune_model.max_len , 1)
# out_size = data.labels_df.shape[1]
# hid_size = finetune_model.d_model 
# finetune_model.predict_layer2 = nn.Linear(hid_size , out_size)

In [42]:


optimizer = torch.optim.AdamW(finetune_model.parameters(), lr=1e-3)

# Finetune Training Loop

In [43]:
i = 0
max_epoch = 200
best_loss = 1e10
best_finetune_model = copy.deepcopy(best_model)
best_epoch = 0
device = "cuda"
finetune_model.to(device)
while i < max_epoch:
    train_loss = []
    progress_bar = tqdm(train_dataloader)
    
    for IDs in progress_bar:
        finetune_model.train()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.reshape(-1, max_len, X.shape[-1])
        targets = torch.tensor(data.labels_df.loc[IDs].to_numpy()).to(device)
        pred = finetune_model.predict(X.float()).squeeze(-1)
        loss = loss_fn(pred, targets)


        optimizer.zero_grad()
        loss.backward()

        nn.utils.clip_grad_norm_(finetune_model.parameters(), max_norm=4.0)
        optimizer.step()

        progress_bar.set_description("Epoch {} - Training loss: {}".format(i, loss)) 
        train_loss.append(loss)
    
    val_loss = []
    for IDs in val_dataloader:
        finetune_model.eval()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.reshape(-1, max_len, X.shape[-1])
        targets = torch.tensor(data.labels_df.loc[IDs].to_numpy()).to(device)
        pred = finetune_model.predict(X.float()).squeeze(-1)
        loss = loss_fn(pred, targets)
        val_loss.append(loss)
    
    train_loss = torch.tensor(train_loss).mean()
    val_loss = torch.tensor(val_loss).mean()
    
    if val_loss < best_loss:
        best_loss = val_loss
        best_finetune_model = copy.deepcopy(finetune_model)
        best_epoch = i
    
    progress_bar.write("Epoch {} - Training loss: {} - Validation loss: {}".format(i, train_loss, val_loss))
    i += 1
    
    
progress_bar.write("Best Epoch {} - Best Validation loss: {}".format(best_epoch, best_loss))

Epoch 1 - Training loss: 48352.125:   1%|▌                                          | 2/149 [00:00<00:07, 18.76it/s]

Epoch 0 - Training loss: 31492.59765625 - Validation loss: 28988.14453125


Epoch 2 - Training loss: 28452.390625:   1%|▌                                       | 2/149 [00:00<00:08, 17.46it/s]

Epoch 1 - Training loss: 27886.349609375 - Validation loss: 23294.662109375


Epoch 3 - Training loss: 14163.3779296875:   2%|▋                                   | 3/149 [00:00<00:06, 21.59it/s]

...

Epoch 197 - Training loss: 3701.179443359375 - Validation loss: 2687.814453125


Epoch 199 - Training loss: 4150.63525390625:   2%|▋                                 | 3/149 [00:00<00:06, 21.47it/s]

Epoch 198 - Training loss: 3658.193359375 - Validation loss: 2344.675048828125


Epoch 199 - Training loss: 2626.169677734375: 100%|███████████████████████████████| 149/149 [00:06<00:00, 21.58it/s]


Epoch 199 - Training loss: 3792.5830078125 - Validation loss: 2514.562744140625
Best Epoch 161 - Best Validation loss: 2311.41943359375


In [44]:
test_loss = []
for IDs in test_dataloader:
    best_finetune_model.eval()
    X = torch.tensor(test_data.feature_df.loc[IDs].to_numpy()).to(device)
    X = X.reshape(-1, max_len, X.shape[-1])
    targets = torch.tensor(test_data.labels_df.loc[IDs].to_numpy()).to(device)
    pred = best_finetune_model.predict(X.float()).squeeze(-1)
    loss = loss_fn(pred, targets)
    test_loss.append(loss)


test_loss = torch.tensor(test_loss).mean()
print("Test loss: {}".format(test_loss))
print("Test loss: {}".format(np.sqrt(test_loss)))

Test loss: 3786.213623046875
Test loss: 61.5322151184082


# Forecasting Results from the paper

<center><img src="../img/img_8.PNG"/><center/>

Reference:
1. https://arxiv.org/abs/2302.00861
2. https://github.com/gzerveas/mvts_transformer

BeijingPM25Quality

No Pretrain

Test loss: 4083.34765625

Test loss: 4096.765625

Test loss: 3880.41064453125

Test loss: 3818.454833984375

Test loss: 3640.204833984375

Test loss: 3793.255615234375

Test loss: 3386.54638671875

Test loss: 3532.69140625

Test loss: 3568.943115234375

Test loss: 3639.838623046875

Pretrain

Test loss: 3398.438720703125

Test loss: 3389.167724609375

Test loss: 3470.773681640625

Test loss: 3443.25830078125

Test loss: 3331.67919921875

Test loss: 3753.1474609375

Test loss: 3480.992431640625

Test loss: 3532.7275390625

Test loss: 3068.515869140625

Test loss: 3118.0390625

Hyperparameter

config['normalization_layer'] = 'BatchNorm'

config['out_len'] = 1

config['out_dim'] = 1

config['d_model'] = 8

config['dim_feedforward'] = 64

config['num_heads'] = 4

config['num_layers'] = 2

No Pretrain

Test loss: 3492.47705078125

Test loss: 3782.492431640625

Test loss: 3879.20458984375

Test loss: 3876.313232421875

Test loss: 3542.3046875

Test loss: 3592.865966796875

Pretrain L//2

Test loss: 3254.694091796875

Test loss: 3107.428955078125

Test loss: 3612.37939453125

Test loss: 3387.132568359375

Test loss: 3596.321533203125

Test loss: 3533.36083984375

Test loss: 3259.4228515625

Test loss: 3758.171630859375

Test loss: 3398.135986328125

Test loss: 3677.972412109375

Pretrain L//4

Test loss: 3581.83984375

Test loss: 3993.34814453125

Test loss: 3416.56103515625

Test loss: 3612.152587890625

Test loss: 3512.76025390625

Test loss: 3326.46044921875

Test loss: 3355.169677734375

Test loss: 3313.980712890625

Test loss: 3644.6640625

Test loss: 3978.413818359375

Hyperparameter

config['normalization_layer'] = 'BatchNorm'

config['out_len'] = 1

config['out_dim'] = 1

config['d_model'] = 8

config['dim_feedforward'] = 32

config['num_heads'] = 4

config['num_layers'] = 2

No Pretrain

Test loss: 3380.054931640625

Test loss: 3759.367919921875